In [1]:
import numpy as np

e = "temp.c,3 humidity,2 co2.ppm,2 voc.ppm,1"
d = "Manifold_hot:20.2224 Manifold_cold1:18.1245 Manifold_cold2:17.1234 5':83 10':72 5':221 10':268 voc:50"

key = e.split()
values = d.split()

h = ['value','label','event','units']

def j():
    json_body = [
            {
                "measurement": measure,
                "tags": {
                    "label": var,
                    "location": location,
                    "name": name
                },
                "time": curr_time,
                "fields": {
                    "value": val
                }
            }
        ]
    return json_body

In [2]:
# will pass the event and the data
def interpret_result(key,values):
    _x = np.empty([len(values),len(h)], dtype='S128')
    _x[:,0] = np.array(values)

    return _x

In [3]:
# Returns a subset of a column of strings
# split(x[:,0], ":", 0)
def split_col(array, in_column, delim, result_index=1):
    y = []
    for i in array[:, in_column]:
        y.append(i.split(delim)[result_index])
    return y

In [6]:
def gen_units(arr):

    new_arr = np.empty([len(arr),0])
    return new_arr

In [16]:
def interpret_values(key, values):
    z = np.array(values, dtype="a128").reshape([len(values),1])
    z = np.append(z, np.array(split_col(z, 0, ":")).reshape([len(z),1]), axis=1 )
    z[:,0] = np.array(split_col(z, 0, ":", 0))
    return z

In [22]:
def gen_key_arr(key, values):
    _l = []
    for _k in key:
        _l.append(_k.split(","))
        _n = np.array(_l)
    
    new_arr = np.empty([0,2], dtype="S32")
    
    for counter, r in enumerate(_n):
        new_arr = np.append(new_arr, np.repeat([_n[counter]], int(_n[counter,1]), axis=0), axis=0)
    return new_arr

In [23]:
# %timeit np.append(interpret_values(key, values), gen_key_arr(key, values), axis=1)

In [98]:
v = np.append(interpret_values(key, values), gen_key_arr(key, values), axis=1)
v

array([['Manifold_hot', '20.2224', 'temp.c', '3'],
       ['Manifold_cold1', '18.1245', 'temp.c', '3'],
       ['Manifold_cold2', '17.1234', 'temp.c', '3'],
       ["5'", '83', 'humidity', '2'],
       ["10'", '72', 'humidity', '2'],
       ["5'", '221', 'co2.ppm', '2'],
       ["10'", '268', 'co2.ppm', '2'],
       ['voc', '50', 'voc.ppm', '1']], 
      dtype='|S128')

In [99]:
# Returns cleaned EVENT column, UNIT column
def parse_events(arr):
    # these will hold our lists
    events = []
    units = []
    
    for i in arr[:,2:4][:,0]:
        _s = i.split(".")
        
        # add events
        try: 
            events.append(_s[0])
        except:
            event.append("error")
        
        # add units (if possible)
        try: 
            units.append(_s[1])
        except:
            units.append("None")
    
    events = np.array(events)
    units = np.array(units)
    return events, units

event_col = parse_events(v)[0]
unit_col = parse_events(v)[1]

In [101]:
v[:,2] = event_col
v[:,3] = unit_col

In [102]:
v

array([['Manifold_hot', '20.2224', 'temp', 'c'],
       ['Manifold_cold1', '18.1245', 'temp', 'c'],
       ['Manifold_cold2', '17.1234', 'temp', 'c'],
       ["5'", '83', 'humidity', 'None'],
       ["10'", '72', 'humidity', 'None'],
       ["5'", '221', 'co2', 'ppm'],
       ["10'", '268', 'co2', 'ppm'],
       ['voc', '50', 'voc', 'ppm']], 
      dtype='|S128')

In [104]:
#### Correct output! 